### Fine tuning & deploying Flan-T5-Large to Amazon Bedrock using Custom Model Import (Using PEFT + FSDP)

this notebook was tested on a ["g5.12xlarge"](https://aws.amazon.com/ec2/instance-types/g5/) instance. 

This notebook will use the HuggingFace Transformers library to fine tune FLAN-T5-Large. due to the fine tuning process being "local" you can run this notebook anywhere as long as the proper compute is available. 

1. Configuring an AWS EC2 instance with a Deep Learning AMI, and setting up a Jupyter Server: [Link](https://docs.aws.amazon.com/dlami/latest/devguide/launch-config.html)
2. Configuring an Amazon Sagemaker environment: [Link](https://docs.aws.amazon.com/sagemaker/latest/dg/gs.html)
3. Configure your own environment, with equivalent compute

This notebook covers the step by step process of fine tuning a [FLAN-T5 large](https://huggingface.co/google/flan-t5-large) mode and deploying it using Bedrock's [Custom Model Import](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-import-model.html) feature. 
The fine tuning data we will be using is based on medical terminology this data can be found on HuggingFace [here](https://huggingface.co/datasets/gamino/wiki_medical_terms). In many discplines, industry specific jargon is used and an LLM may not have the correct understanding of words, context or abbreviations. By fine tuning a model on medical terminology in this case, the LLM is given the ability to understand specific jargon and answer questions the user might have. 

The resulting files are imported into Amazon Bedrock via custom model import

WARNING: This method of Custom Model Import will only work with "FLAN-t5-large". "FLAN-t5-small" is incompatible with Bedrock Custom Model Import in its current state. This is due to the number of heads in the model needing to be a multiple of 4, due to the model needing to be sharded accordingly in the GPU. the number of heads for FLAN-t5-small is 6. this can be checked in the model's config.json file under the parameter "num_heads" 

### Installs & Imports 

we will be utilizing HuggingFace Transformers library to pull a pretrained model from the Hub and fine tune it. The dataset we will be finetuning on will also be pulled from HuggingFace

In [ ]:
%%capture
%pip install transformers --quiet
%pip install torch
%pip install -U bitsandbytes accelerate transformers peft trl
%pip install datasets --quiet
%pip install sentencepiece --quiet

In [ ]:
import bitsandbytes
import torch
from transformers import BitsAndBytesConfig, T5ForConditionalGeneration, T5Tokenizer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
import accelerate
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
import torch

### Pull a pre-trained model from HuggingFace 

as mentioned at the beginning of the notebook, we will be fine tuning google's FLAN-t5-large from HuggingFace. This model is free to pull - no HuggingFace account needed.

The model is loaded with the "bitsandbytes" library. This allows the model to be quantized in 4-bit, to set it up for fine tuning with FSDP-QLoRA

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

#Set Model Attributes

model_name = "google/flan-t5-large"

model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)
tokenizer = T5Tokenizer.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

### Pull a dataset from HuggingFace 

The dataset we are pulling can be looked at [here](https://huggingface.co/datasets/gamino/wiki_medical_terms). This is a dataset containing over 6000+ medical terms, and their wiki definitions. 

Since the model is being trained to recognize & understand medical terminology, with the function below the dataset will be converted to a Q & A format. "What is" is added as a prefix to the medical terms column, and the other column stays as the definition.

One important aspect here is ensuring the "padding=True" argument is passed. This is needed when training a FLAN-T5 model with FSDP + QLoRA

In [ ]:
#Load data from huggingface
ds = load_dataset("gamino/wiki_medical_terms")

#Using 70% of the dataset to train
ds = ds["train"].train_test_split(test_size=0.3)

#Process data to fit a Q & A format 
prefix = "What is "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # Transform page title into an answer:
   inputs = [prefix + doc for doc in examples["page_title"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding=True)
  
   # keep explanations as is:
   labels = tokenizer(text_target=examples["page_text"], 
                      max_length=512, truncation=True, padding=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

# Map the preprocessing function across our dataset
ds = ds.map(preprocess_function, batched=True)
#Take the training portion of the dataset
ds = ds["train"]

### Set up training parameters

In the first cell below, the PEFT configurations are being set up. The term PEFT has been mentioned a couple of times in this notebook already, but what is it? PEFT stands for Parameter Efficient Fine Tuning. It allows the majority of the model parameters to be frozen, and only fine tune a small number of them. This technique decreases computation & storage costs, without performance suffering. Read more about HuggingFace's PEFT library [here](https://huggingface.co/docs/peft/en/index) 

In the second all training parameters are being passed into the "SFTTrainer" class. This is powered by the accelerate library. It allows for FSDP - Fully Sharded Data Parallel. At a high level this training technique, shards a model's parameters across the number of available GPU's. Read more about HuggingFace's FSDP library [here](https://huggingface.co/docs/transformers/main/en/fsdp)  

Combining these techniques can lead to cost effective, high performance fine tuning for models. Read more about combining PEFT & FSDP on HuggingFace [here](https://huggingface.co/docs/bitsandbytes/main/en/fsdp_qlora)

Note: Training is set to 1 epoch. This is for a faster training time to showcase Bedrock Custom Model Import. If you require fine tuning with higher performance, consider increasing the epochs & optimizing hyperparameters passed

In [ ]:
#Set up PEFT Configurations 
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    target_modules="all-linear",
)

In [ ]:
#Pass all parameters to SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    data_collator=data_collator,
    args= TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size = 1,
        num_train_epochs=1,
        gradient_accumulation_steps=2,
        eval_accumulation_steps=2
    ),
)

### Start model training

In the first cell we empty the pytorch cache to free as much memory on the device possible. In the next cell, "trainer.train()" actually stars the training job.

In the training parameters passed, the line output_dir="./results" saves the model checkpoints into the "results" folder in the device directory (creates it if not already created). The final model is in this directory as "checkpoint-XXX" - XXX being the largest number. 

This training job will take approx. 30 mins 

In [ ]:
#Empty pytorch cache
torch.cuda.empty_cache()

In [ ]:
trainer.train()

### Model Inference 

We will now take our latest checkpoint and generate text with it

In [ ]:
#Model Inference 
last_checkpoint = "./results/checkpoint-600" #Load checkpoint that you want to test 

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

med_term = "what is Dexamethasone suppression test" 

query = tokenizer(med_term, return_tensors="pt")
output = finetuned_model.generate(**query, max_length=512, no_repeat_ngram_size=True)
answer = tokenizer.decode(output[0])

print(answer)

### Model Upload 

with out model now generating text related to medical terminology we will now upload it to S3 to ensure readiness for Bedrock Custom Model Import. Depending on the environment you have chosen to run this notebook in, your AWS credentials will have to be initialized to upload the model files to your S3 bucket of choice. 

In [ ]:
#Upload model to S3 Bucket
import boto3
import os
# Set up S3 client
s3 = boto3.client('s3')

# Specify your S3 bucket name and the prefix (folder) where you want to upload the files
bucket_name = 'your-bucket-here'#YOU BUCKET HERE
model_name = "results/checkpoint-#" #YOUR LATEST CHECKPOINT HERE (this will be in the "results" folder in your notebook directory replace the "#" with the latest checkpoint number)
prefix = 'flan-t5-large-medical/' + model_name

# Upload files to S3
def upload_directory_to_s3(local_directory, bucket, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(s3_prefix, relative_path)
            
            print(f'Uploading {local_path} to s3://{bucket}/{s3_path}')
            s3.upload_file(local_path, bucket, s3_path)

# Call the function to upload the downloaded model files to S3
upload_directory_to_s3(model_name, bucket_name, prefix)

### Importing Model to Amazon Bedrock

Now that our model artifacts are uploaded into an S3 bucket, we can import it into Amazon Bedrock 

in the AWS console, we can go to the Amazon Bedrock page. On the left side under "Foundation models" we will click on "Imported models"

![Step 1](./images/step1.png "Step 1")


You can now click on "Import model"

![Step 2](./images/step2.png "Step 2")

In this next step you will have to configure:

1. Model Name 
2. Import Job Name 
3. Model Import Settings 
    a. Select Amazon S3 bucket 
    b. Select your bucket location (uploaded in the previous section)
4. Create a IAM role, or use an existing one (not shown in image)
5. Click Import (not shown in image)

![Step 3](./images/step3.png "Step 3")


You will now be taken to the page below. Your model may take up to an hour to import. 

![Step 4](./images/step4.png "Step 4")

After your model imports you will then be able to test it via the playground or API! 

![Playground](./images/playground.gif "Playground")

END OF NOTEBOOK